# Biologics Pharmacology AI/ML Workflow Notebook

This notebook provides an interactive, end-to-end walkthrough for building and validating data-driven models to support early decisions in biologics drug development.

## What this notebook covers

1. Generate (or load) integrated synthetic biologics data
2. Validate schema and inspect key variables
3. Run grouped cross-validation by molecule family
4. Train a final multi-output model for PK, PD, and safety
5. Rank molecules with a transparent decision score

> You can replace synthetic data with real preclinical/clinical data once available.

In [ ]:
from pathlib import Path
import sys

import joblib
import pandas as pd
from IPython.display import display

PROJECT_ROOT = Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from biologics_pharmacology.decision_support import rank_candidates
from biologics_pharmacology.modeling import (
    run_grouped_cross_validation,
    train_final_model,
)
from biologics_pharmacology.schema import feature_columns, validate_dataset_schema
from scripts.generate_synthetic_biologics_data import generate_dataset

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

## 1) Load or generate biologics dataset

The code below will reuse `data/synthetic_biologics.csv` if it already exists. Otherwise, it generates a new synthetic dataset.

In [ ]:
data_path = Path("data/synthetic_biologics.csv")
data_path.parent.mkdir(parents=True, exist_ok=True)

if data_path.exists():
    df = pd.read_csv(data_path)
    source = "loaded from existing CSV"
else:
    df = generate_dataset(n_samples=500, seed=42)
    df.to_csv(data_path, index=False)
    source = "generated and saved"

validate_dataset_schema(df)

print(f"Dataset source: {source}")
print(f"Rows: {len(df)} | Columns: {len(df.columns)}")
print(f"Unique molecule families: {df['molecule_family'].nunique()}")

df.head()

## 2) Quick profile of key variables

This gives a high-level summary across modalities, therapeutic areas, and model targets.

In [ ]:
target_columns = [
    "clinical_pk_half_life_day",
    "clinical_pd_response_pct",
    "severe_ae_rate_pct",
]

print("Counts by modality:")
display(df["modality"].value_counts().to_frame("count"))

print("\nCounts by therapeutic area:")
display(df["therapeutic_area"].value_counts().to_frame("count"))

print("\nTarget summary:")
display(df[target_columns].describe().round(3))

## 3) Grouped cross-validation

We evaluate with `GroupKFold` on `molecule_family` to reduce leakage across related molecules.

In [ ]:
cv_metrics = run_grouped_cross_validation(df, n_splits=5)

cv_summary = (
    cv_metrics.groupby("target")[["mae", "r2"]]
    .agg(["mean", "std"])
    .round(3)
)

print("Fold-level sample:")
display(cv_metrics.head())

print("\nGrouped CV summary by target:")
display(cv_summary)

## 4) Train final model on all data

After validation, train a final model artifact for downstream decision support.

In [ ]:
artifacts = train_final_model(df)

model_path = Path("models/biologics_multitask_model.joblib")
model_path.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(artifacts, model_path)

print(f"Saved model artifacts to: {model_path}")

## 5) Rank candidate biologics for early development decisions

The score favors:
- higher predicted PD response
- longer predicted PK half-life
- lower predicted severe AE rate

In [ ]:
candidate_table = df[["molecule_id"] + feature_columns()].copy()

top_candidates = rank_candidates(
    artifacts=artifacts,
    candidates=candidate_table,
    top_k=15,
)

top_candidates[
    [
        "molecule_id",
        "pred_clinical_pk_half_life_day",
        "pred_clinical_pd_response_pct",
        "pred_severe_ae_rate_pct",
        "decision_score",
    ]
].round(3)

## 6) Adapting this notebook to real R&D data

To move from synthetic to real biologics programs:

1. Replace `df` with your governed integrated dataset.
2. Keep required schema columns from `biologics_pharmacology/schema.py`.
3. Preserve grouped validation by molecule family (or program-level grouping).
4. Tune model settings and decision-score weights with QP and clinical stakeholders.
5. Add uncertainty estimates before deployment into portfolio decisions.